# Download libraries if required

In [0]:
%pip install monai[fire]==1.0.0 torchvision pytorch-ignite==0.4.11 autogen-agentchat<=0.3 
%pip install openai

In [0]:
dbutils.library.restartPython()

In [0]:
# !pip install gdown
!python -m monai.bundle download "breast_density_classification" --version "0.1.0" --bundle_dir "./"
!cp -r  breast_density_classification/models ../models
!rm -rf breast_density_classification 
!rm -rf breast_density_classification_v0.1.0.zip
# !python -m monai.bundle download -h

# Import Libraries

In [0]:
from typing import Annotated, Literal
import mlflow
from monai.bundle import run
from monai.bundle.config_parser import ConfigParser
import pandas as pd


# Get classification for an Mammogram Image using Monai Model Zoo 

In [0]:


def get_breast_density_classification(image_url : Annotated[str, "the local path of the image"]) -> dict:

    d= {"data": {
    "_target_": "createList.CreateImagefromJson",
    "J": {"image" : image_url}}}
    config_file = "../configs/inference_new.json"
    parser = ConfigParser()    
    parser.read_config(config_file)
    run(config_file=config_file, workflow_type="inference",**d)

    data = parser.get_parsed_content("output_dir")
    df = pd.read_csv(f"{data}/predictions.csv",header=None)
    return {"Output": chr(ord('@')+df[[1,2,3,4]].idxmax('columns')[0])}


In [0]:
image_path = "../sample_data/D/sample_D3.jpg"
get_breast_density_classification(image_path)

In [0]:
# Load Secret if running locally else get in-context secret
from dotenv import load_dotenv
from pathlib import Path
import os

dotenv_path = Path('.env')
load_dotenv(dotenv_path=dotenv_path)
# 


In [0]:
import base64
from openai import OpenAI
from mlflow.utils.databricks_utils import get_databricks_host_creds

workspace_host = "https://adb-984752964297111.11.azuredatabricks.net"
endpoint = f"{workspace_host}/serving-endpoints"
token = get_databricks_host_creds().token 

# Initialize the OpenAI client with the Databricks serving endpoint
client = OpenAI(
    base_url=endpoint,
    api_key=token
)



# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Getting the base64 string
base64_image = encode_image(image_path)

response = client.chat.completions.create(
  model="intern-38b",
  messages=[
    {"role": "user", 
    "content": [
                {
                    "type": "image_url",
                    "image_url": {
                      "url": "https://www.arsenal.com/sites/default/files/styles/large_16x9/public/images/saka-celeb-bayern.png?h=3c8f2bed&auto=webp&itok=Twjeu8tug"
                    },
                },
            ],
    },
    # {"role": "user", 
    # "content": [
    #             {
    #                 "type": "text",
    #                 "text": "What's the weather like in San Francisco?"
            
    # }
    # ]
    # }
  ],
  tools = [{
    "type": "function",
    "function": {
        "name": "get_player_information",
        "description": "This functions gets all details about a player shown in the image including where he is male/female and club affiliation",
        "parameters": {
            "type": "object",
            "properties": {
                "team": {
                    "type":
                    "string",
                    "description":
                    "The sports club or team the player belongs to can be from any sport"
                },
                "gender": {
                    "type":
                    "string",
                    "description":
                    "The gender of the player can be either male or female or 'no player found'",
                },
            },
            "required": ["team", "gender"]
        }
    }
},
{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {"type": "string", "description": "City and state, e.g., 'San Francisco, CA'"},
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]}
            },
            "required": ["location", "unit"]
        }
    }
}
],
# tool_choice={"type": "function", "function": {"name": "get_player_information"}},
  temperature=0.1,
  max_tokens=200,

)

print(response)

In [0]:
import json
import os
import random
import time
from typing import Any, Callable, Dict, List, Optional, Tuple, Type, Union

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from termcolor import colored

import autogen
from autogen import Agent, AssistantAgent, ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.capabilities.vision_capability import VisionCapability
from autogen.agentchat.contrib.img_utils import get_pil_image, pil_to_data_uri
from autogen.code_utils import content_str

In [0]:
llm_config = {
    "config_list": [
        {
            "model": "intern-38b",
            "api_key": token,
            "base_url": endpoint,
        }
    ],
    "temperature": 0.5,
    "max_tokens": 500
}

In [0]:
import os

from autogen import ConversableAgent
from autogen.agentchat.contrib.multimodal_conversable_agent import MultimodalConversableAgent

# Let's first define the assistant agent that suggests tool calls.
assistant_p = MultimodalConversableAgent(
    name="image-explainer",
    max_consecutive_auto_reply=10,
    llm_config={"config_list": llm_config['config_list']},
    system_message= "you are an mammogram image analyser your job is to classify the breast density based on the mammogram provided use the tools in your you have specified and give explanation about the implications"
    "Return 'TERMINATE' when the task is done."
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

# Register the tool signature with the assistant agent.
assistant_p.register_for_llm(name="get_breast_density_classification", 
                           description="This a function which takes an input path which can be local path and gives classification of Density of the mammogram")(get_breast_density_classification)

# Register the tool function with the user proxy agent.
user_proxy.register_for_execution(name="get_breast_density_classification")(get_breast_density_classification)

In [0]:
from autogen import register_function

# Register the calculator function to the two agents.
register_function(
    get_breast_density_classification,
    caller=assistant_p,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="get_breast_density_classification",  # By default, the function name is used as the tool name.
    description="This a function which takes an input path which can be local path and gives classification of Density of the mammogram",  # A description of the tool.
)

In [0]:
# Ask the question with an image
# open method used to open different extension image file
# Imports PIL module
from PIL import Image
im = Image.open(image_path)

user_proxy.initiate_chat(
    assistant_p,
    message = {

                "content": 
                    [   {"type": "text","text" : f" local path of the mammogram  {image_path}"},
                        {"type": "image_url", "image_url": {"url": im}},
                        # {"type": "text","text": " classify the density of the breast?"}
                        ],
            }
)